# 6° Desafio de Dados de Moedas Digitais

Risco DeFi-Nativo" (Aave) com o "Risco TradFi-Tokenizado" (OUSG da Ondo Finance)

O Aave é um dos maiores e mais antigos protocolos de lending (empréstimo) descentralizados. Ele não lida com "crédito privado" (empréstimos para empresas), mas sim com empréstimos sobrecolateralizados (usuários depositam, ex: ETH, para pegar emprestado USDC). O seu "ativo de risco" seria o yield pago para quem deposita USDC na plataforma.

O OUSG é um token (RWA) que representa a posse de uma cota em um fundo de Títulos do Tesouro Americano (T-Bills) de curto prazo, gerido pela Ondo. O yield (juros) é pago aos detentores do token através de um mecanismo chamado rebasing (você vê seu número de tokens OUSG aumentar diariamente).

Estamos querendo modelar o spread de juros de rendimento de ambos os ativos. Isto é

$$ Y = Yield_{Aave} - Yield_{Ondo} $$

Tese: O spread de juros​ não é aleatório. Ele é um "termômetro de risco" do sistema DeFi, e podemos modelar seus principais vetores (drivers) on-chain e off-chain.

(melhorar)

In [15]:
import os
import pandas as pd


## Obtendo os dados

Peguem suas API_KEY

In [16]:
from dotenv import load_dotenv
from dune_client.client import DuneClient

load_dotenv()

api_key = os.getenv("DUNE_API_KEY")
dune = DuneClient(api_key)

result = dune.get_latest_result(1950532)

In [17]:
df = pd.DataFrame(result.result.rows)

print(df.head())
print(df.info())
print(df.describe())

print(f"Total de linhas: {len(df)}")
print(f"Colunas: {df.columns.tolist()}")

# ??? Por algum motivo ta puxando de BTC

             BTC  Total_Borrow_volume  Total_Borrowers  Total_Borrows  \
0  105983.002755         5.723234e+08              954           1453   
1  103364.049246         3.358354e+09             4639           9839   
2  111578.744633         2.936805e+09             4236           8094   
3  110418.339717         2.954168e+09             3640           7268   
4  110248.861443         3.299789e+09             4223           8632   

   Total_Depositors  Total_Deposits  Total_Deposits_volume  \
0              1677            2701           2.070275e+09   
1             10721           21745           9.962585e+09   
2              6688           12938           7.586627e+09   
3              6293           12824           1.095289e+10   
4              7608           15950           9.182817e+09   

   Total_Repay_volume  Total_Repayers  Total_Repays  Total_Withdraw_users  \
0        4.367196e+08             464          1168                   885   
1        3.889908e+09            4

In [18]:
df.to_csv('aave_dune_data.csv', index=False)

In [19]:
# Definir o nome do arquivo de entrada
input_filename = '../data/aave/aavescan-aave-v3-ethereum-usdc-history.csv'
# Definir o nome do arquivo de saída
output_filename = 'aave_fase2_cleaned.csv'

try:
    # Carregar o DataFrame
    df = pd.read_csv(input_filename)

    print(f"Carregado {input_filename}. Iniciando a Fase 2...")

    # --- FASE 2: LIMPEZA ---

    # 1. Tratar valores nulos (com base na observação anterior do .info())
    df.dropna(inplace=True)
    print("- Linhas com valores nulos removidas.")

    # 2. Padronização Temporal: Converter 'date' para datetime
    df['date'] = pd.to_datetime(df['date'])
    print("- Coluna 'date' convertida para datetime.")

    # 3. Ordenar: Garantir a ordem cronológica correta (mais antigo -> mais novo)
    df.sort_values(by='date', ascending=True, inplace=True)
    print("- DataFrame ordenado por data (ascendente).")

    # 4. Definir 'date' como o índice do DataFrame
    # Isso é crucial para operações de séries temporais (rolling, diff)
    df.set_index('date', inplace=True)
    print("- Coluna 'date' definida como índice.")

    # --- FASE 2: ENGENHARIA DE FEATURES ---
    # (Aplicada às colunas que temos neste arquivo)

    # 1. Renomear a coluna alvo (Y1) para clareza, como no roteiro
    if 'supply APR' in df.columns:
        df.rename(columns={'supply APR': 'aave_usdc_apy'}, inplace=True)
        print("- Coluna 'supply APR' renomeada para 'aave_usdc_apy'.")
    
    # 2. Criar Taxa de Utilização (X_onchain)
    if 'borrowed' in df.columns and 'supplied' in df.columns:
        df['aave_utilization_rate'] = df['borrowed'] / df['supplied']
        print("- Feature criada: 'aave_utilization_rate' (borrowed / supplied).")
        
        # 3. Criar Aceleração da Utilização (X_onchain), como pedido no roteiro
        # .diff() calcula a diferença do dia anterior
        df['aave_utilizacao_aceleracao'] = df['aave_utilization_rate'].diff()
        print("- Feature criada: 'aave_utilizacao_aceleracao' (.diff()).")

    # 4. Criar Risco de De-peg (X_onchain), como identificado na análise
    if 'price' in df.columns:
        df['usdc_depeg_risk'] = (1.0 - df['price']).abs()
        print("- Feature criada: 'usdc_depeg_risk' (1.0 - price).")

    # --- SAÍDA ---

    print("\n### DataFrame final (head) após a Fase 2 ###")
    # O primeiro dia terá NaN para 'aave_utilizacao_aceleracao', o que é normal
    print(df.head())

    print("\n### Informações (info) do DataFrame final ###")
    df.info()

    # 5. Salvar o DataFrame processado em um novo CSV
    df.to_csv(output_filename)
    
    print(f"\nSucesso! O DataFrame limpo e processado foi salvo como: {output_filename}")

except FileNotFoundError:
    print(f"Erro: O arquivo '{input_filename}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento: {e}")

Carregado ../data/aave/aavescan-aave-v3-ethereum-usdc-history.csv. Iniciando a Fase 2...
- Linhas com valores nulos removidas.
- Coluna 'date' convertida para datetime.
- DataFrame ordenado por data (ascendente).
- Coluna 'date' definida como índice.
- Coluna 'supply APR' renomeada para 'aave_usdc_apy'.
- Feature criada: 'aave_utilization_rate' (borrowed / supplied).
- Feature criada: 'aave_utilizacao_aceleracao' (.diff()).
- Feature criada: 'usdc_depeg_risk' (1.0 - price).

### DataFrame final (head) após a Fase 2 ###
                            timestamp symbol  \
date                                           
2025-08-14 00:00:00+00:00  1755129600   USDC   
2025-08-15 00:00:00+00:00  1755216000   USDC   
2025-08-16 00:00:00+00:00  1755302400   USDC   
2025-08-17 00:00:00+00:00  1755388800   USDC   
2025-08-18 00:00:00+00:00  1755475200   USDC   

                                                              address  \
date                                                             